In [1]:
import tensorflow as tf
from sklearn import metrics, cross_validation
import pandas as pd
import numpy as np
from numpy import genfromtxt

In [2]:
X_data = pd.read_csv('EURUSD15.csv', sep=',', names=['Time', 'Open', 'Max', 'Min', 'Close', 'Value'])

In [3]:
ii = 0
y = []
for i in X_data.Open:
    if (X_data['Close'].values[ii] > i):
        y.append(1)
    else:
        y.append(0)
    ii+=1


In [4]:
def to_vector(y,clas =  2):
    num_labels = len(y)
    index_off = np.arange(num_labels)*2
    labels_one_hot = np.zeros((num_labels, clas))
    labels_one_hot.flat[index_off + y] = 1
    return labels_one_hot
del X_data['Time']

In [5]:
y_one_hot = to_vector(y, 2)
X_train, X_test, y_train, y_test = cross_validation.train_test_split(X_data.values, y_one_hot, test_size=0.2, random_state=42)
y_one_hot

array([[ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       ..., 
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [6]:
# Parameters
learning_rate = 0.05
batch_size = 150
display_step = 1

# Network Parameters
n_hidden_1 = 50 # 1st layer number of features
n_hidden_2 = 20 # 2nd layer number of features
n_input = 5 # data
n_classes = 2 #total classes (0-1 digits)

In [7]:
# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [8]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [9]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
n_samples = list(X_train.shape)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [10]:
sess = tf.Session()
init = tf.initialize_all_variables()
sess.run(init)

In [11]:
for i in range(100):
    average_cost = 0
    number_of_bathes = int(len(X_train) / batch_size)
    for start, end in zip(range(0, len(X_train), batch_size), range(batch_size, len(X_train), batch_size)):
        sess.run(optimizer, feed_dict={x: X_train[start:end], y: y_train[start:end]})
        average_cost += sess.run(cost, feed_dict = {x: X_train[start:end], y: y_train[start:end]})
    print("Epoch:", '%04d' % (i+1), 'cost', "{:.9f}".format(average_cost))
print("Finished optimization!")
    

('Epoch:', '0001', 'cost', '82446.598791897')
('Epoch:', '0002', 'cost', '662.014433980')
('Epoch:', '0003', 'cost', '239.803106010')
('Epoch:', '0004', 'cost', '239.814270973')
('Epoch:', '0005', 'cost', '239.824503839')
('Epoch:', '0006', 'cost', '239.833875120')
('Epoch:', '0007', 'cost', '239.843899548')
('Epoch:', '0008', 'cost', '239.856371880')
('Epoch:', '0009', 'cost', '239.872519672')
('Epoch:', '0010', 'cost', '239.892417312')
('Epoch:', '0011', 'cost', '239.914475620')
('Epoch:', '0012', 'cost', '239.936218441')
('Epoch:', '0013', 'cost', '239.955777884')
('Epoch:', '0014', 'cost', '239.972387314')
('Epoch:', '0015', 'cost', '239.985823870')
('Epoch:', '0016', 'cost', '239.996176600')
('Epoch:', '0017', 'cost', '240.003803194')
('Epoch:', '0018', 'cost', '240.009245396')
('Epoch:', '0019', 'cost', '240.013026953')
('Epoch:', '0020', 'cost', '240.015617192')
('Epoch:', '0021', 'cost', '240.017386496')
('Epoch:', '0022', 'cost', '240.018574238')
('Epoch:', '0023', 'cost', '24

In [12]:
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y_test, 1))

In [13]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
print('Accuracy: {0}'.format(sess.run(accuracy, feed_dict={x:X_test, y: y_test})))

Accuracy: 0.503384590149
